# Semantic検索


ML環境構築

1. Amazon BedrockとのMLコネクターを作成
1. モデルグループを作成
1. モデルを作成
1. モデルをデプロイ
1. Ingestパイプラインを作成
1. Searchパイプラインを作成

インデックス作成

1. インデックスを作成
1. データを登録

検索

1. キーワード検索
1. ニューラル検索
1. ハイブリッド検索

---
## OpenSearch 起動・停止

* 起動

In [ ]:
%%bash
docker compose -f "../0_opensearch-docker/docker-compose.yml" up -d


* 停止

In [66]:
%%bash
docker compose -f "../0_opensearch-docker/docker-compose.yml" down
# docker compose -f "../0_opensearch-docker/docker-compose.yml" down -v


 Container opensearch-dashboards  Stopping
 Container opensearch-node1  Stopping
 Container opensearch-dashboards  Stopped
 Container opensearch-dashboards  Removing
 Container opensearch-dashboards  Removed
 Container opensearch-node1  Stopped
 Container opensearch-node1  Removing
 Container opensearch-node1  Removed
 Network 0_opensearch-docker_opensearch-net  Removing
 Network 0_opensearch-docker_opensearch-net  Removed


---
## 事前準備

In [4]:
%pip install -Uq requests python-dotenv


Note: you may need to restart the kernel to use updated packages.


In [7]:
import os
import requests


In [6]:
from dotenv import load_dotenv

load_dotenv(override=True)


True

In [3]:
host = 'localhost'
port = 9200


---
## 1. Amazon BedrockとのMLコネクターを作成

In [8]:
body = {
  "name": "Amazon Bedrock Connector: embedding",
  "description": "The connector to the Bedrock Titan embedding model",
  "version": 1,
  "protocol": "aws_sigv4",
  "parameters": {
    "region": os.getenv("AWS_DEFAULT_REGION"),
    "service_name": "bedrock"
  },
  "credential": {
    "access_key": os.getenv("AWS_ACCESS_KEY_ID"),
      "secret_key": os.getenv("AWS_SECRET_ACCESS_KEY")
  },
  "actions": [
    {
      "action_type": "predict",
      "method": "POST",
      "url": "https://bedrock-runtime.us-east-1.amazonaws.com/model/amazon.titan-embed-text-v1/invoke",
      "headers": {
        "content-type": "application/json",
        "x-amz-content-sha256": "required"
      },
      "request_body": "{ \"inputText\": \"${parameters.inputText}\" }",
      "pre_process_function": "\n    StringBuilder builder = new StringBuilder();\n    builder.append(\"\\\"\");\n    String first = params.text_docs[0];\n    builder.append(first);\n    builder.append(\"\\\"\");\n    def parameters = \"{\" +\"\\\"inputText\\\":\" + builder + \"}\";\n    return  \"{\" +\"\\\"parameters\\\":\" + parameters + \"}\";",
      "post_process_function": "\n      def name = \"sentence_embedding\";\n      def dataType = \"FLOAT32\";\n      if (params.embedding == null || params.embedding.length == 0) {\n        return params.message;\n      }\n      def shape = [params.embedding.length];\n      def json = \"{\" +\n                 \"\\\"name\\\":\\\"\" + name + \"\\\",\" +\n                 \"\\\"data_type\\\":\\\"\" + dataType + \"\\\",\" +\n                 \"\\\"shape\\\":\" + shape + \",\" +\n                 \"\\\"data\\\":\" + params.embedding +\n                 \"}\";\n      return json;\n    "
    }
  ]
}

response = requests.post(
  f'http://{host}:{port}/_plugins/_ml/connectors/_create', 
  json=body
  )

response.json()


{'connector_id': 'I-w-w4sBPX3Z0k6U9jOi'}

In [10]:
connector_id = response.json()["connector_id"]
connector_id


'I-w-w4sBPX3Z0k6U9jOi'

---
## 2. モデルグループを作成


In [11]:
model_group_name = "bedrock-embedded-model-group"

body = {
    "name": model_group_name
}

response = requests.post(
  f'http://{host}:{port}/_plugins/_ml/model_groups/_register', 
  json=body
  )

response.json()


{'model_group_id': 'JOxBw4sBPX3Z0k6UTzMb', 'status': 'CREATED'}

In [12]:
model_group_id = response.json()["model_group_id"]
model_group_id


'JOxBw4sBPX3Z0k6UTzMb'

---
## 3. モデルを作成


In [13]:
model_name = "bedrock-embedded-model"

body = {
    "name": model_name,
    "function_name": "remote",
    "model_group_id": model_group_id,
    "connector_id": connector_id
}

response = requests.post(
  f'http://{host}:{port}/_plugins/_ml/models/_register', 
  json=body
  )

response.json()


{'task_id': 'JexCw4sBPX3Z0k6UzTMe',
 'status': 'CREATED',
 'model_id': 'JuxCw4sBPX3Z0k6U0jPD'}

In [14]:
model_id = response.json()["model_id"]
model_id


'JuxCw4sBPX3Z0k6U0jPD'

---
## 4. モデルをデプロイ


In [15]:
response = requests.post(
  f'http://{host}:{port}/_plugins/_ml/models/{model_id}/_deploy', 
  )

response.json()


{'task_id': 'J-xDw4sBPX3Z0k6U0TNW',
 'task_type': 'DEPLOY_MODEL',
 'status': 'COMPLETED'}

---
## 5. Ingestパイプラインを作成


In [54]:
ingest_pipeline_name = "nlp-ingest-pipeline"
embedding_field = "passage_embedding"
embedding_target_field = "question"

body = {
  "description": "An NLP ingest pipeline",
  "processors": [
    {
      "text_embedding": {
        "model_id": model_id,
        "field_map": {
          embedding_target_field: embedding_field
        }
      }
    }
  ]
}

response = requests.put(
  f'http://{host}:{port}/_ingest/pipeline/{ingest_pipeline_name}', 
  json=body
  )

response.json()


{'acknowledged': True}

---
## 6. Searchパイプラインを作成


In [17]:
search_pipeline_name = 'nlp-search-pipeline'

body = {
  "description": "Post processor for hybrid search",
  "phase_results_processors": [
    {
      "normalization-processor": {
        "normalization": {
          "technique": "min_max"
        },
        "combination": {
          "technique": "arithmetic_mean",
          "parameters": {
            "weights": [
              0.3,
              0.7
            ]
          }
        }
      }
    }
  ]
}

response = requests.put(
  f'http://{host}:{port}/_search/pipeline/{search_pipeline_name}', 
  json=body
  )

response.json()


{'acknowledged': True}

---
## 1. インデックスを作成


In [18]:
index_name = 'semantic-index'


In [56]:
body = {
  "settings": {
    "index.knn": True,
    "default_pipeline": ingest_pipeline_name,
    "index": {
      "analysis": {
        "analyzer": {
          "custom_kuromoji_analyzer": {
            "tokenizer": "kuromoji_tokenizer",
            "filter": ["kuromoji_baseform", "ja_stop"],
            "char_filter": ["icu_normalizer"]
          }
        }
      }
    }
  },
  "mappings": {
    "properties": {
      embedding_field: {
        "type": "knn_vector",
        "dimension": 1536
      },
      "question": {"type": "text", "analyzer": "custom_kuromoji_analyzer"},
      "answer": {"type": "text", "analyzer": "custom_kuromoji_analyzer"}
    }
  }
}

response = requests.put(
  f"http://{host}:{port}/{index_name}", 
  json=body
  )

response.json()


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'semantic-index'}

---
## 2. データを登録

In [22]:
%pip install -Uq langchain beautifulsoup4


Note: you may need to restart the kernel to use updated packages.


In [23]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import CharacterTextSplitter

loader = WebBaseLoader("https://aws.amazon.com/jp/ec2/faqs/")
data = loader.load()

text_splitter = CharacterTextSplitter(
    separator = "Q:",
    keep_separator=True,
    chunk_size = 10,
    chunk_overlap  = 0,
)

# 先頭のいらないものを消す
texts = text_splitter.split_documents(data)
texts = texts[1:]
# 末尾のいらないものを消す
texts[-1].page_content = texts[-1].page_content.split('\xa0')[0]


Created a chunk of size 1283, which is longer than the specified 10
Created a chunk of size 154, which is longer than the specified 10
Created a chunk of size 425, which is longer than the specified 10
Created a chunk of size 275, which is longer than the specified 10
Created a chunk of size 190, which is longer than the specified 10
Created a chunk of size 582, which is longer than the specified 10
Created a chunk of size 832, which is longer than the specified 10
Created a chunk of size 483, which is longer than the specified 10
Created a chunk of size 203, which is longer than the specified 10
Created a chunk of size 826, which is longer than the specified 10
Created a chunk of size 211, which is longer than the specified 10
Created a chunk of size 574, which is longer than the specified 10
Created a chunk of size 322, which is longer than the specified 10
Created a chunk of size 199, which is longer than the specified 10
Created a chunk of size 365, which is longer than the specifi

In [57]:
for text in texts:
  try:
    lines = text.page_content.splitlines()
    q = lines[0]
    a = text.page_content

    body = {
      "question": q,
      "answer": a
    }

    response = requests.post(
      f'http://{host}:{port}/{index_name}/_doc', 
      json=body
    )
  except Exception as e:
    print(e)


---
## 1. キーワード検索

In [62]:
question = "EC2で起動できるOSは何ですか？"


In [63]:
body = {
  "_source": {
    "excludes": [
      embedding_field
    ]
  },
  "query": {
    "match": {
      "question": {
        "query": question
      }
    }
  }
}

response = requests.get(
  f"http://{host}:{port}/{index_name}/_search", 
  json=body
  )

response.json()


{'took': 13,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 205, 'relation': 'eq'},
  'max_score': 6.3473625,
  'hits': [{'_index': 'semantic-index',
    '_id': 'Uexnw4sBPX3Z0k6UtTW3',
    '_score': 6.3473625,
    '_source': {'question': 'Q: Amazon EC2 で何ができますか?',
     'answer': 'Q: Amazon EC2 で何ができますか?\nAmazon Simple Storage Service (Amazon S3) がクラウド内のストレージを可能とするのとまったく同様に、Amazon EC2 は、クラウド内での「コンピューティング」を可能にします。\xa0 Amazon EC2 のシンプルなウェブサービスインターフェイスによって、手間をかけず、必要な機能を取得および設定できます。お客様のコンピューティングリソースに対して、高機能なコントロールが提供され、Amazon の実績あるインフラストラクチャ上で実行できます。Amazon EC2 では、わずか数分間で新規サーバーインスタンスを取得して起動できるようになります。これにより、コンピューティング要件の変化に合わせて、すばやく容量をスケールアップおよびスケールダウンできます。実際に使用した分のみ料金が発生するため、Amazon EC2 はコンピューティングの経済性も変革します。'}},
   {'_index': 'semantic-index',
    '_id': 'Sexqw4sBPX3Z0k6UmTd8',
    '_score': 6.303764,
    '_source': {'question': 'Q:Nitro Hypervisor では、インスタンスの再起動と終了の EC2 API リクエストがどのように実装されますか?',
     'answer': 'Q:Nitro Hy

---
## 2. ニューラル検索

In [64]:
body = {
  "_source": {
    "excludes": [
      embedding_field
    ]
  },
  "query": {
    "neural": {
      embedding_field: {
        "query_text": question,
        "model_id": model_id,
      }
    }
  }
}

response = requests.get(
  f"http://{host}:{port}/{index_name}/_search", 
  json=body
  )

response.json()


{'took': 285,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 33, 'relation': 'eq'},
  'max_score': 0.008197499,
  'hits': [{'_index': 'semantic-index',
    '_id': 'Uuxnw4sBPX3Z0k6UtzWJ',
    '_score': 0.008197499,
    '_source': {'question': 'Q: どうすれば Amazon EC2 の使用を開始できますか?',
     'answer': 'Q: どうすれば Amazon EC2 の使用を開始できますか?\nAmazon EC2 にサインアップするには、Amazon EC2 詳細ページ上の [このウェブサービスにサインアップ] ボタンを選択します。このサービスにアクセスするには、AWS のアカウントを保有している必要があります。これをまだ持っていない場合は、Amazon EC2 サインアッププロセスの開始時に、プロンプト画面が表示されてこれを作成することができます。サインアップの後、Amazon EC2 ドキュメントをご参照ください。ここには開始方法に関するガイドが含まれています。'}},
   {'_index': 'semantic-index',
    '_id': 'Uuxqw4sBPX3Z0k6Upjec',
    '_score': 0.0077156513,
    '_source': {'question': 'Q: 既存の Windows Server ライセンスを EC2 で使用できますか?',
     'answer': 'Q: 既存の Windows Server ライセンスを EC2 で使用できますか?\nはい、できます。ImportImage ツールを使用して自分の Windows Server マシンイメージをインポートした後、それらのマシンイメージから EC2 Dedicated Hosts にインスタンスを起動して、インスタンスを効果的に管理

---
## 3. ハイブリッド検索


In [65]:
body = {
  "_source": {
    "excludes": [
      "passage_embedding"
    ]
  },
  "query": {
    "hybrid": {
      "queries": [
        {
          "match": {
            "question": {
              "query": question
            }
          }
        },
        {
          "neural": {
            "passage_embedding": {
              "query_text": question,
              "model_id": model_id,
            }
          }
        }
      ]
    }

  }
}

response = requests.get(
  f"http://{host}:{port}/{index_name}/_search", 
  json=body
  )

response.json()


{'took': 301,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 205, 'relation': 'eq'},
  'max_score': 6.3473625,
  'hits': [{'_index': 'semantic-index',
    '_id': 'Uexnw4sBPX3Z0k6UtTW3',
    '_score': -9549511700.0,
    '_source': {'question': 'Q: Amazon EC2 で何ができますか?',
     'answer': 'Q: Amazon EC2 で何ができますか?\nAmazon Simple Storage Service (Amazon S3) がクラウド内のストレージを可能とするのとまったく同様に、Amazon EC2 は、クラウド内での「コンピューティング」を可能にします。\xa0 Amazon EC2 のシンプルなウェブサービスインターフェイスによって、手間をかけず、必要な機能を取得および設定できます。お客様のコンピューティングリソースに対して、高機能なコントロールが提供され、Amazon の実績あるインフラストラクチャ上で実行できます。Amazon EC2 では、わずか数分間で新規サーバーインスタンスを取得して起動できるようになります。これにより、コンピューティング要件の変化に合わせて、すばやく容量をスケールアップおよびスケールダウンできます。実際に使用した分のみ料金が発生するため、Amazon EC2 はコンピューティングの経済性も変革します。'}},
   {'_index': 'semantic-index',
    '_id': 'Uexnw4sBPX3Z0k6UtTW3',
    '_score': -4422440400.0,
    '_source': {'question': 'Q: Amazon EC2 で何ができますか?',
     'answer': 'Q: Amazon EC2 で何ができますか?\nAmazon Simple St

---
## クリーンアップ

* インデックス削除

In [55]:
response = requests.delete(
  f"http://{host}:{port}/{index_name}", 
  )

response.json()


{'acknowledged': True}